In [1]:
import sys
sys.path.append('..')

import torch

import lib

In [2]:
from lib.constants import HYPERPARAMETERS

params = HYPERPARAMETERS
params['batch_size'] = 1
params['n_epochs'] = 10
params['verbose'] = True
params['version'] = '0.1'

if not params['disable_cuda'] and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
params['device'] = device

for p in params:
    print(f"{p} : \t{params[p]}")

batch_size : 	1
checkpoints_dir : 	/home/sergevkim/git/FacesClassification/checkpoints
checkpoint_filename : 	
disable_cuda : 	False
imgs_dir : 	/home/sergevkim/git/FacesClassification/data/CelebaHQ
label : 	Male
labels_filename : 	/home/sergevkim/git/FacesClassification/data/list_attr_celeba.txt
logs_dir : 	/home/sergevkim/git/FacesClassification/logs
n_epochs : 	10
n_imgs : 	30000
verbose : 	True
version : 	0.1
device : 	cuda:0


In [3]:
import numpy as np
from sklearn.metrics import accuracy_score


def train_phase(model, train_loader, criterion, optimizer, device, epoch, verbose=False):
    model.train()
    loss_history = []

    for batch_idx, batch in enumerate(train_loader):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs).double() #outputs is from 0 to 1
        outputs_2 = outputs * 2 - 1 #is it differentiate?

        optimizer.zero_grad()
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

        if verbose:
            if batch_idx % 100 == 0:
                print('train', batch_idx, loss.item())

        loss_history.append(loss.item())

    print(f"{epoch}: done!")
    return loss_history


def valid_phase(model, valid_loader, device, epoch, verbose=False):
    model.eval()
    accuracy_scores_history = []

    for batch_idx, batch in enumerate(valid_loader):
        inputs, labels = batch
        inputs = inputs.to(device)
        outputs = model(inputs).double() #outputs is from 0 to 1
        outputs = outputs.cpu().detach().numpy()
        # do have inputs be cuda-like if model is cuda-like?
        predicts = outputs.round() * 2 - 1
        #predicts.cpu()?

        batch_accuracy = accuracy_score(predicts, labels)

        accuracy_scores_history.append(batch_accuracy)

    return accuracy_scores_history

In [ ]:
from torch.nn import BCELoss
from torch.optim import Adam

from lib.models import ResNet
from lib.utils import train_parse_args
from lib.utils.data import get_data_loaders



N_EPOCHS = 10

loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'],
    label=params['label'])
train_loader = loaders['train_loader']
valid_loader = loaders['valid_loader']

model = ResNet().to(device)
criterion = BCELoss()
optimizer = Adam(model.parameters(), lr=3e-4)

'''
accuracy_scores_history = valid_phase(
    model=model,
    valid_loader=valid_loader,
    device=device,
    epoch=0)
'''

for epoch in range(N_EPOCHS):
    print(f"EPOCH: {epoch}")
    loss_history = train_phase(
        model=model,
        train_loader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epoch=epoch,
        verbose=True)
    accuracy_scores_history = valid_phase(
        model=model,
        valid_loader=valid_loader,
        device=device,
        epoch=epoch,
        verbose=True)
    avg_valid_accuracy = np.mean(np.array(accuracy_scores_history))
    print(avg_valid_accuracy)

EPOCH: 0


/home/sergevkim/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


train 0 0.1538991152061585
train 100 -1.5416062668453696
train 200 -3.599171088190458
train 300 9.273326333770317
train 400 16.470804322702566
train 500 -27.527740411353864
train 600 -18.44717025610011
train 700 -30.59137345770687
train 800 -42.38199229607343
train 900 -36.760303496061695
train 1000 -44.87329105272902
train 1100 -30.90599439365328
train 1200 -31.987195987486245
train 1300 -34.42033388739677
train 1400 34.38559717030953
train 1500 -46.56291573555032
train 1600 39.53678500275602
train 1700 41.623565725812924
train 1800 -29.31692509509219
train 1900 48.765808196672445
train 2000 60.68291473922667
train 2100 53.133899750336624
train 2200 -55.00442505564123
train 2300 -42.127765653939285
train 2400 -42.938808381827194
train 2500 -49.586856896695366
train 2600 -53.99350730758507
train 2700 56.30299375414248
train 2800 -55.62794497129882
train 2900 -58.97510150764208
train 3000 66.9883956904567
train 3100 33.30948254172417
train 3200 -31.79667853270235
train 3300 -62.81065367